In [ ]:
#installer
%pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

# Create a list with the filepaths for training and testing
data_path = 'input'
train_dir = Path(data_path + '/train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path(data_path + '/test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path(data_path + 'validation')
val_filepaths = list(test_dir.glob(r'**/*.jpg'))

def process_imgs(filepath):
    labels = [Path(filepath[i]).parent.name for i in range(len(filepath))]
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle 
    df = df.sample(frac=1).reset_index(drop = True)
    
    return df

train_dt = process_imgs(train_filepaths)
test_dt = process_imgs(test_filepaths)
val_dt = process_imgs(val_filepaths)